In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.00001
epochs = 1
batch_size = 128

test_valid_size = 256

n_classes = 10
dropout = 0.75

In [3]:
# weights and biases

weights = {
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [4]:
def conv2d(x, w, b, strides = 1):
    x = tf.nn.conv2d(x, w, strides=[1,strides, strides, 1], padding = 'SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

In [5]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1,k,k,1],
        strides=[1,k,k,1],
        padding='SAME'
    )

In [6]:
def conv_net(x, weights, biases, dropout):
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k =2)

    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [11]:
#examples

x =tf.constant([1,2,3,4,5,6,7,8,9])

y = tf.reshape(x, [3,3])

z = tf.reshape(y,[-1,9])
with tf.Session() as sess:
    print(sess.run(z))


[[1 2 3 4 5 6 7 8 9]]


In [12]:
x = tf.placeholder(tf.float32, [None, 28,28,1])
y = tf.placeholder(tf.float32, [None, n_classes])

In [13]:

keep_prob = tf.placeholder(tf.float32)

In [14]:

logits = conv_net(x, weights, biases, keep_prob)

In [15]:

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

In [16]:
correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [17]:

init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples // batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict= {
                x: batch_x,
                y: batch_y,
                keep_prob: dropout
            })

            loss = sess.run(cost, feed_dict ={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.
            })

            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[: test_valid_size],
                y: mnist.validation.labels[: test_valid_size],
                keep_prob :1.
            })

            print('Epoch {: >2} Batch {:>3} -' 
                   'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                       epoch+1,
                       batch +1,
                       loss,
                       valid_acc
                   ))
            
        test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.
    })

    print('Testing Accuracy : {}'.format(test_acc))

Epoch  1 Batch   1 -Loss: 48263.4688 Validation Accuracy: 0.167969
Epoch  1 Batch   2 -Loss: 38277.8359 Validation Accuracy: 0.183594
Epoch  1 Batch   3 -Loss: 35438.6992 Validation Accuracy: 0.207031
Epoch  1 Batch   4 -Loss: 30904.0898 Validation Accuracy: 0.214844
Epoch  1 Batch   5 -Loss: 29866.6953 Validation Accuracy: 0.214844
Epoch  1 Batch   6 -Loss: 26305.1758 Validation Accuracy: 0.222656
Epoch  1 Batch   7 -Loss: 24791.4883 Validation Accuracy: 0.218750
Epoch  1 Batch   8 -Loss: 22067.0371 Validation Accuracy: 0.222656
Epoch  1 Batch   9 -Loss: 22030.4648 Validation Accuracy: 0.242188
Epoch  1 Batch  10 -Loss: 20493.1953 Validation Accuracy: 0.273438
Epoch  1 Batch  11 -Loss: 20607.8008 Validation Accuracy: 0.253906
Epoch  1 Batch  12 -Loss: 18043.3594 Validation Accuracy: 0.265625
Epoch  1 Batch  13 -Loss: 19681.9180 Validation Accuracy: 0.300781
Epoch  1 Batch  14 -Loss: 19803.9902 Validation Accuracy: 0.296875
Epoch  1 Batch  15 -Loss: 19789.8242 Validation Accuracy: 0.32

Epoch  1 Batch 124 -Loss:  4220.7905 Validation Accuracy: 0.656250
Epoch  1 Batch 125 -Loss:  4673.8643 Validation Accuracy: 0.664062
Epoch  1 Batch 126 -Loss:  3795.3647 Validation Accuracy: 0.664062
Epoch  1 Batch 127 -Loss:  5200.3574 Validation Accuracy: 0.656250
Epoch  1 Batch 128 -Loss:  3260.6614 Validation Accuracy: 0.660156
Epoch  1 Batch 129 -Loss:  4168.1626 Validation Accuracy: 0.656250
Epoch  1 Batch 130 -Loss:  4848.0947 Validation Accuracy: 0.656250
Epoch  1 Batch 131 -Loss:  4196.9946 Validation Accuracy: 0.656250
Epoch  1 Batch 132 -Loss:  5076.2266 Validation Accuracy: 0.660156
Epoch  1 Batch 133 -Loss:  3907.3042 Validation Accuracy: 0.656250
Epoch  1 Batch 134 -Loss:  3874.1360 Validation Accuracy: 0.656250
Epoch  1 Batch 135 -Loss:  3615.2854 Validation Accuracy: 0.664062
Epoch  1 Batch 136 -Loss:  3611.0781 Validation Accuracy: 0.656250
Epoch  1 Batch 137 -Loss:  4473.3115 Validation Accuracy: 0.648438
Epoch  1 Batch 138 -Loss:  4778.2710 Validation Accuracy: 0.64

Epoch  1 Batch 247 -Loss:  2673.3550 Validation Accuracy: 0.699219
Epoch  1 Batch 248 -Loss:  2044.1113 Validation Accuracy: 0.703125
Epoch  1 Batch 249 -Loss:  1889.7922 Validation Accuracy: 0.699219
Epoch  1 Batch 250 -Loss:  2902.3997 Validation Accuracy: 0.699219
Epoch  1 Batch 251 -Loss:  2294.5825 Validation Accuracy: 0.703125
Epoch  1 Batch 252 -Loss:  2470.2314 Validation Accuracy: 0.695312
Epoch  1 Batch 253 -Loss:  2615.0049 Validation Accuracy: 0.691406
Epoch  1 Batch 254 -Loss:  2016.0645 Validation Accuracy: 0.695312
Epoch  1 Batch 255 -Loss:  2625.6841 Validation Accuracy: 0.699219
Epoch  1 Batch 256 -Loss:  3061.2683 Validation Accuracy: 0.703125
Epoch  1 Batch 257 -Loss:  2414.1772 Validation Accuracy: 0.707031
Epoch  1 Batch 258 -Loss:  2236.9502 Validation Accuracy: 0.714844
Epoch  1 Batch 259 -Loss:  2093.3057 Validation Accuracy: 0.703125
Epoch  1 Batch 260 -Loss:  1520.0679 Validation Accuracy: 0.695312
Epoch  1 Batch 261 -Loss:  1865.9021 Validation Accuracy: 0.70

Epoch  1 Batch 370 -Loss:  1939.9888 Validation Accuracy: 0.710938
Epoch  1 Batch 371 -Loss:  1355.8149 Validation Accuracy: 0.714844
Epoch  1 Batch 372 -Loss:  1364.4653 Validation Accuracy: 0.707031
Epoch  1 Batch 373 -Loss:  1391.2898 Validation Accuracy: 0.714844
Epoch  1 Batch 374 -Loss:  1654.0654 Validation Accuracy: 0.710938
Epoch  1 Batch 375 -Loss:  1849.6780 Validation Accuracy: 0.703125
Epoch  1 Batch 376 -Loss:  1383.4188 Validation Accuracy: 0.710938
Epoch  1 Batch 377 -Loss:  2375.2024 Validation Accuracy: 0.710938
Epoch  1 Batch 378 -Loss:  2270.8025 Validation Accuracy: 0.710938
Epoch  1 Batch 379 -Loss:  1389.7944 Validation Accuracy: 0.714844
Epoch  1 Batch 380 -Loss:  2151.7866 Validation Accuracy: 0.707031
Epoch  1 Batch 381 -Loss:  2045.3518 Validation Accuracy: 0.703125
Epoch  1 Batch 382 -Loss:  2062.0869 Validation Accuracy: 0.707031
Epoch  1 Batch 383 -Loss:  1353.7397 Validation Accuracy: 0.714844
Epoch  1 Batch 384 -Loss:   914.3492 Validation Accuracy: 0.71

Epoch  2 Batch  64 -Loss:  1073.6954 Validation Accuracy: 0.718750
Epoch  2 Batch  65 -Loss:  1456.5059 Validation Accuracy: 0.730469
Epoch  2 Batch  66 -Loss:  1436.5830 Validation Accuracy: 0.726562
Epoch  2 Batch  67 -Loss:  1636.1865 Validation Accuracy: 0.730469
Epoch  2 Batch  68 -Loss:  1357.1184 Validation Accuracy: 0.734375
Epoch  2 Batch  69 -Loss:  1519.2994 Validation Accuracy: 0.738281
Epoch  2 Batch  70 -Loss:  1334.3905 Validation Accuracy: 0.734375
Epoch  2 Batch  71 -Loss:  1440.3053 Validation Accuracy: 0.726562
Epoch  2 Batch  72 -Loss:   639.5417 Validation Accuracy: 0.730469
Epoch  2 Batch  73 -Loss:  1039.5579 Validation Accuracy: 0.726562
Epoch  2 Batch  74 -Loss:  1089.5000 Validation Accuracy: 0.726562
Epoch  2 Batch  75 -Loss:  1664.7754 Validation Accuracy: 0.726562
Epoch  2 Batch  76 -Loss:   997.1065 Validation Accuracy: 0.730469
Epoch  2 Batch  77 -Loss:   954.3126 Validation Accuracy: 0.738281
Epoch  2 Batch  78 -Loss:  1215.3939 Validation Accuracy: 0.73